In [5]:
import numpy as np
import gymnasium as gym
from scipy.linalg import solve_continuous_are
import time

# Define the dynamics and cost functions
def dynamics(x, u):
    """
    Dynamics of the Continuous Mountain Car environment.
    x: state vector [position, velocity]
    u: control input [force]
    """
    position, velocity = x
    force = np.clip(u, -1.0, 1.0)  # Clip force to valid range
    new_velocity = velocity + 0.001 * force - 0.0025 * np.cos(3 * position)
    new_position = position + new_velocity
    new_velocity = np.clip(new_velocity, -0.07, 0.07)  # Clip velocity to valid range
    return np.array([new_position, new_velocity])

def cost(x, u):
    """
    Cost function for the Continuous Mountain Car problem.
    x: state vector [position, velocity]
    u: control input [force]
    """
    position, velocity = x
    target_position = 0.45  # Target position (top of the hill)
    return (position - target_position) ** 2 + 0.1 * (u ** 2)  # Quadratic cost

def linearize_dynamics(x, u):
    """
    Linearize the dynamics around the current state and control.
    Returns A (state Jacobian) and B (control Jacobian).
    """
    position, velocity = float(x[0]), float(x[1])  # Ensure scalars
    A = np.array([[1, 0.001], [0.0025 * 3 * np.sin(3 * position), 1]], dtype=np.float64)
    B = np.array([[0], [0.001]], dtype=np.float64)
    return A, B

def iLQR(env, max_iter=100, tol=1e-6):
    """
    Iterative Linear Quadratic Regulator (iLQR) algorithm.
    """
    # Initialize state and control
    state = env.reset()[0]
    u = np.zeros(1)  # Initial control input

    # Define cost matrices
    Q = np.diag([1.0, 0.1])  # State cost
    R = np.array([[0.1]])  # Control cost

    for _ in range(max_iter):
        # Linearize dynamics
        A, B = linearize_dynamics(state, u)

        # Solve Riccati equation for the value function
        P = solve_continuous_are(A, B, Q, R)

        # Compute optimal control
        K = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A
        u_new = K @ state.reshape(-1, 1)

        # Check for convergence
        if np.linalg.norm(u_new - u) < tol:
            break

        # Update control
        u = u_new

        # Simulate one step
        state = dynamics(state, u)

    return u

# Main function to run the iLQR algorithm
def main():
    env = gym.make("MountainCarContinuous-v0")
    optimal_control = iLQR(env)
    print("Optimal Control Input:", optimal_control)

    # Test the optimal control
    state = env.reset()[0]
    total_cost = 0
    for _ in range(1000):
        env.render()
        #time.sleep(0.01)  # Slow down for visibility
        state = dynamics(state, optimal_control)
        total_cost += cost(state, optimal_control)
        if state[0] >= 0.45:  # Check if the car reached the goal
            print("Goal reached!")
            break
    print("Total Cost:", total_cost)
    env.close()

if __name__ == "__main__":
    main()

Optimal Control Input: [[296011.57180216]]
Total Cost: [[8.76228506e+12]]


C:\Users\ganga\AppData\Local\Temp\ipykernel_6328\4182227672.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  position, velocity = float(x[0]), float(x[1])  # Ensure scalars


In [6]:
env = gym.make("MountainCarContinuous-v0")
state = env.reset()[0]
print(state)

[-0.53583056  0.        ]


In [ ]:
import numpy as np
import gymnasium as gym
from scipy.linalg import solve_continuous_are

# Define the dynamics and cost functions
def dynamics(x, u):
    """
    Dynamics of the Continuous Mountain Car environment.
    x: state vector [position, velocity]
    u: control input [force]
    """
    position, velocity = x
    force = np.clip(u, -1.0, 1.0)  # Clip force to valid range
    new_velocity = velocity + 0.001 * force - 0.0025 * np.cos(3 * position)
    new_position = position + new_velocity
    new_velocity = np.clip(new_velocity, -0.07, 0.07)  # Clip velocity to valid range
    return np.array([new_position, new_velocity])

def cost(x, u):
    """
    Cost function for the Continuous Mountain Car problem.
    x: state vector [position, velocity]
    u: control input [force]
    """
    position, velocity = x
    target_position = 0.45  # Target position (top of the hill)
    return (position - target_position) ** 2 + 0.1 * (u ** 2)  # Quadratic cost

def linearize_dynamics(x, u):
    """
    Linearize the dynamics around the current state and control.
    Returns A (state Jacobian) and B (control Jacobian).
    """
    position, velocity = x
    A = np.array([[1, 0.001], [0.0025 * 3 * np.sin(3 * position), 1]])
    B = np.array([[0], [0.001]])
    return A, B

def iLQR(env, max_iter=100, tol=1e-6):
    """
    Iterative Linear Quadratic Regulator (iLQR) algorithm.
    """
    # Initialize state and control
    state = env.reset()[0]
    u = np.zeros(1)  # Initial control input

    # Define cost matrices
    Q = np.diag([1.0, 0.1])  # State cost
    R = np.array([[0.1]])  # Control cost

    for _ in range(max_iter):
        # Linearize dynamics
        A, B = linearize_dynamics(state, u)

        # Solve Riccati equation for the value function
        P = solve_continuous_are(A, B, Q, R)

        # Compute optimal control
        K = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A
        u_new = K @ state

        # Check for convergence
        if np.linalg.norm(u_new - u) < tol:
            break

        # Update control
        u = u_new

        # Simulate one step
        state = dynamics(state, u)

    return u

# Main function to run the iLQR algorithm
def main():
    env = gym.make("MountainCarContinuous-v0")
    optimal_control = iLQR(env)
    print("Optimal Control Input:", optimal_control)

    # Test the optimal control
    state = env.reset()[0]
    total_cost = 0
    for _ in range(1000):
        env.render()
        state = dynamics(state, optimal_control)
        total_cost += cost(state, optimal_control)
        if state[0] >= 0.45:  # Check if the car reached the goal
            print("Goal reached!")
            break
    print("Total Cost:", total_cost)
    env.close()

if __name__ == "__main__":
    main()